In [125]:
from pydriller import Repository
import csv
import os.path
import pandas as pd
from jira import JIRA

In [126]:
repoPath = 'https://github.com/apache/ActiveMQ'
commitsFile = 'commits.csv'
commits = []

if not(os.path.isfile(commitsFile)) :
    with open(commitsFile, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['hash', 'msg', 'deletions', 'insertions'])
        writer.writeheader()                        
        for commit in Repository(repoPath).traverse_commits():
            record = {'hash': commit.hash, 'msg': commit.msg, 'deletions': commit.deletions, 'insertions': commit.insertions}
            writer.writerow(record)

In [127]:
df = pd.read_csv(commitsFile, dtype={"hash": "string", "msg": "string", "deletions": "int", "insertions": "int"})
df.head()

,hash,msg,deletions,insertions
0,40a7d3b6ac35d2ecb34e85fc3403d2e48e33874e,Moved the trunk code into the trunk sub direct...,0,194375
1,8f1763f078525b3cbfd30dd8389d3f61da56ac78,Moved the trunk code into the trunk sub direct...,0,899
2,fba1488453499bddd675dbbb04054703044b4526,updated the build so that the recursive build ...,31,42
3,8f40a7eeca8772a16e45a15ef424142e4f90e129,removed old configuration documentation git-s...,7,0
4,ab8f458612b8c3020d7d0446216083c17221d257,removed old configuration documentation git-s...,1,0


In [128]:
jira = JIRA(server="https://issues.apache.org/jira")

def get_all_issues(jira_client, project_name, fields):
    issues = []
    i = 0
    chunk_size = 100
    while True:
        
        chunk = jira_client.search_issues(f'project = {project_name}', startAt=i, maxResults=chunk_size, fields=fields)
        i += chunk_size
        issues += chunk.iterable
        if i >= chunk.total:
            break
    return issues

In [129]:
import warnings
warnings.filterwarnings('ignore')

jiraIssuesFile = 'jiraIssues.csv'

if not(os.path.isfile(jiraIssuesFile)) :
    issues = get_all_issues(jira, 'ActiveMQ', ['id', 'resolution', 'status', 'issuetype', 'description'])
    
    with open(jiraIssuesFile, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['id', 'key', 'status', 'issuetype', 'resolution', 'description'])
        writer.writeheader()                        
        for issue in issues:
            if issue.raw['fields']['issuetype']['name'] == 'Bug':
                
                record = {'id': issue.id, 'key': issue.key, 
                          'status': issue.raw['fields']['status']['name'], 
                          'issuetype': issue.raw['fields']['issuetype']['name'],
                          'resolution': "None" if (issue.raw['fields']['resolution'] is None) else issue.raw['fields']['resolution']['name'],
                          'description': issue.raw['fields']['description']
                         }
                writer.writerow(record)

In [55]:
jira.issue_type(1)

<JIRA IssueType: name='Bug', id='1'>